<a href="https://colab.research.google.com/github/buganart/text_to_music/blob/master/gameMusic_CGAN_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description


In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
#@markdown - The directory of training music (.mid) data
data_dir = "/content/text_to_music/data" #@param {type:"string"}
#@markdown - The output directory of generated videos 
out_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-gameMusic_CGAN" #@param {type:"string"}
#@markdown - set wandb run id of logged run to resume from there
resume_id = "" #@param {type:"string"}

#@markdown ##training parameters
seq_length = 100 #@param {type:"integer"}
latent_dim = 1000 #@param {type:"integer"}
lr = 0.0002 #@param {type:"number"}
batch_size = 32 #@param {type:"integer"}

#@markdown ##log parameters
num_samples_per_class = 3 #@param {type:"integer"}
save_interval = 2000 #@param {type:"integer"}

In [ ]:
#@title Clone text_to_music repo

!git clone https://github.com/buganart/text_to_music

clear_on_success("Repo downloaded.")

In [ ]:
#@title Install Dependencies

%cd /content/text_to_music/midi-lstm-gan
# %pip install -r requirements.txt
%pip install matplotlib
%pip install music21==5.7.1
%pip install wandb
%pip install click
clear_on_success("Dependencies installed.")

In [ ]:
#@title TRAIN
# os.environ["WANDB_MODE"] = "dryrun"

!python mlp_gan.py \
--resume_id "$resume_id" \
--exp_path "$out_dir" \
--data_path "$data_dir" \
--seq_length "$seq_length" \
--latent_dim "$latent_dim" \
--lr "$lr" \
--batch_size "$batch_size" \
--num_samples_per_class "$num_samples_per_class" \
--save_interval "$save_interval" 